<a href="https://colab.research.google.com/github/Presage-Security/presage_technologies/blob/main/examples/Physiology_API_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Limitations**

In order to ensure best performance, we recommend that the videos you process meet the following conditions.  Within a 20 second window, it is ideal if there is a single foremost subject in view.  That subject's gaze is mostly unchanged and looking in the general direction of the acquiring camera. The subject’s face, shoulders and chest should be visible.  Well-lit, uniformly-lit, and consistently-lit environments will also ensure optimal performance.  Videos must be acquired at frame rates greater than 10 frames per second. 


#### **Video format**

The current API requires that input videos have a standard, RGB color encoding.  We ensure support for the following formats: AVI, MOV and MP4.  

#### **ISO/Exposure/Gain **

Some cameras automatically adjust exposure time, gain, ISO or f/stop in order to optimize image brightness.  This can be problematic for Presage Vitals if said values vary dramatically. While Presage can deal with varying lighting changes in the recording environment, rapid lighting changes can cause ISO, gain, and exposure to vary significantly and be problematic for proper vital extraction. For optimal results, ensure the recording device is not performing rapid ISO/exposure/gain adjustments or keep the scene illumination relative constant. 

#### **Distance**

Presage technology can measure vitals for a large degree of subject to camera distance.  It only requires that there are at least approximately 100 camera pixels across the face and that said face is in focus.  For a 12 megapixel camera, this is approximately 3-5% of the total width of the image. 

Extreme imaging distances in atmospheric conditions that display visible scintillation in recorded videos can cause errors in the vital extraction. 

#### **Lighting**
While Presage can deal with monochromatic visible light sources for illumination, broadband light sources covering the entire visible spectrum are ideal. 

Scene illumination should not be too dark or too bright. Ensure the average recorded pixel intensity of the subject's face is above 30% of the dynamic range of the recording device without gain being applied. Also, ensure the pixels corresponding to the subject's face are not saturated. Video recorded not following these lighting guidelines could result in no vitals being obtained. 


#### **Motion**
_Camera motion_

Although we expect to handle camera motion in the future, currently large degrees of camera motion will severely impact the ability to generate respiration rate and may impact heart rate output.  For optimal results, maintain a fairly still camera position during each 20 second window in which vital measurements are desired.

_Subject motion_

Similar to camera motion, we have found that subject motion can degrade the ability to reliably generate a respiratory rate measurement and to a lesser degree, a heart rate measurement.  It takes approximately 10 seconds of video with a stationary subject before a new vital measurement is returned. If a subject moves their head significantly, that counter resets.  We recommend the subject remains still, with their gaze pointed towards the acquiring camera. The subject should also refrain from talking as this leads to errors in respiration rate measurements.

# **Code**

In [ ]:
!pip install presage_technologies --quiet
from presage_technologies import Physiology
import time

In [ ]:
#Initialize the Physiology class with your API key. If you don't already have one, you can sign up for the trial here: https://physiology.presagetech.com
print("Please enter an API Key in the box below. If you don't have one yet, sign up for free here: https://physiology.presagetech.com/")
API_KEY = input("API Key: ")
if API_KEY == '':
  print("You must have an API Key to run this tool!")
  exit()
physio = Physiology(str(API_KEY).strip())

In [ ]:
#This is only needed for our Google Colab Demo to download our demo video. If running locally this cell can be removed.
#If you'd like to run locally, check out our documentation here: https://docs.physiology.presagetech.com/#python-client
import os
if not os.path.exists("/content/presage_demo_video.avi"):
  !pip install gdown --quiet
  import gdown
  gdown.download(id="1d3n7_3b54GVVb7-WpkQMSiyp4831P03j", output="/content/presage_demo_video.avi", quiet=True)

In [ ]:
video_id = physio.queue_processing_hr_rr("/content/presage_demo_video.avi")
#We add a manual sleep here as all of our processing is done via serverless containers that are launched on upload.
#The Physiology class has additional sleeps built in should it take longer than the predefined sleep cycle.
#If you have a need for faster processing please reach out to support@presagetech.com to discuss options.
time.sleep(60)
data = physio.retrieve_result(video_id)

In [ ]:
print(data)

In [ ]:
#Used to visualize the results
!pip install matplotlib --quiet
import matplotlib.pylab as plt
import math

rrs = [(math.floor(float(a)), b) for a,b in data["rr"]["values"].items()]
hrs = [(math.floor(float(a)), b) for a,b in data["hr"]["values"].items()]

x_rr, y_rr = zip(*rrs)
x_hr, y_hr = zip(*hrs)

%matplotlib inline
plt.plot(x_hr, y_hr, label = "HR")
plt.legend()
plt.show()
plt.plot(x_rr, y_rr, label = "RR")
plt.legend()
plt.show()